In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user, list_users
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test: One Offs

In [ ]:
mock_user = get_user(105359460)
response = run_new_training_week_process_wrapped(mock_user)
print(response)

In [ ]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

# Evals: New Training Week

In [ ]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

In [ ]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

# Manual: New Training Week

In [11]:
from freezegun import freeze_time
from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.new_training_week import generate_training_week_with_coaching
from src.supabase_client import list_users

@freeze_time("2024-09-01")
def gen_training_week_wrapper(sysmsg_base, strava_client):
    activities_df = get_activities_df(strava_client)
    day_of_week_summaries = get_day_of_week_summaries(activities_df)
    weekly_summaries = get_weekly_summaries(activities_df)
    return generate_training_week_with_coaching(
        sysmsg_base=sysmsg_base,
        weekly_summaries=weekly_summaries,
        day_of_week_summaries=day_of_week_summaries,
    )

# "voynow99@gmail.com", "jaredpalek8@gmail.com", "laurenbrooks01@gmail.com", "raytrip.curtis@gmail.com"
for user in list_users():
    if user.email == "bconnolly04@gmail.com":
        print(user.email)
        sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
        strava_client = get_strava_client(user.athlete_id)
        training_week_with_coaching = gen_training_week_wrapper(sysmsg_base, strava_client)
        print(training_week_with_coaching)
        break

bconnolly04@gmail.com
athlete_id=105359460 token still valid until 2024-09-03 04:28:54+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training schedule shows short runs spread throughout the week, with a longer run on Saturdays averaging 4.87 miles at a pace of about 9 minutes 40 seconds per mile. It seems you rest on Thursdays, and it's advisable to incorporate speed work into your shorter Tuesday or Wednesday runs to improve your half marathon pace., weekly_mileage_target=Based on your recent training history, it seems you're building up nicely, with a peak mileage of 15.86 in Week 34. For the upcoming week, I suggest aiming for a total weekly mileage of around 16-17 miles to maintain consistency without overreaching. Your longest run previously was 12.32 miles, so let's aim for a long run this week of 11-12 miles, scheduled for Saturday. This will help you continue building endurance safely while setting a solid foundation for your half marathon preparation

In [12]:
print(training_week_with_coaching.total_weekly_mileage)
print(training_week_with_coaching.weekly_mileage_target)
training_week_with_coaching.training_week

17.5
Based on your recent training history, it seems you're building up nicely, with a peak mileage of 15.86 in Week 34. For the upcoming week, I suggest aiming for a total weekly mileage of around 16-17 miles to maintain consistency without overreaching. Your longest run previously was 12.32 miles, so let's aim for a long run this week of 11-12 miles, scheduled for Saturday. This will help you continue building endurance safely while setting a solid foundation for your half marathon preparation.


[TrainingSession(day=<Day.MON: 'mon'>, session_type=<SessionType.EASY: 'easy run'>, distance=2.0, notes='Easy pace to start the week'),
 TrainingSession(day=<Day.TUES: 'tues'>, session_type=<SessionType.SPEED: 'speed workout'>, distance=3.0, notes='1mi warm-up, 2x1mi @ 8m 30s pace with 400m rest in between, cool down.'),
 TrainingSession(day=<Day.WED: 'wed'>, session_type=<SessionType.REST: 'rest day'>, distance=0.0, notes='Rest and recover for the week'),
 TrainingSession(day=<Day.THURS: 'thurs'>, session_type=<SessionType.EASY: 'easy run'>, distance=1.5, notes='Light jog, short and easy.'),
 TrainingSession(day=<Day.FRI: 'fri'>, session_type=<SessionType.REST: 'rest day'>, distance=0.0, notes='Prepare for the long run'),
 TrainingSession(day=<Day.SAT: 'sat'>, session_type=<SessionType.LONG: 'long run'>, distance=11.0, notes='Complete at a comfortable pace around 10:00 min/mile, focus on endurance'),
 TrainingSession(day=<Day.SUN: 'sun'>, session_type=<SessionType.REST: 'rest day'>, d

In [13]:
from src.supabase_client import upsert_training_week_with_coaching

upsert_training_week_with_coaching(user.athlete_id, training_week_with_coaching)

APIResponse(data=[{'id': 70, 'athlete_id': 105359460, 'training_week': '[{"day": "mon", "session_type": "easy run", "distance": 2.0, "notes": "Easy pace to start the week"}, {"day": "tues", "session_type": "speed workout", "distance": 3.0, "notes": "1mi warm-up, 2x1mi @ 8m 30s pace with 400m rest in between, cool down."}, {"day": "wed", "session_type": "rest day", "distance": 0.0, "notes": "Rest and recover for the week"}, {"day": "thurs", "session_type": "easy run", "distance": 1.5, "notes": "Light jog, short and easy."}, {"day": "fri", "session_type": "rest day", "distance": 0.0, "notes": "Prepare for the long run"}, {"day": "sat", "session_type": "long run", "distance": 11.0, "notes": "Complete at a comfortable pace around 10:00 min/mile, focus on endurance"}, {"day": "sun", "session_type": "rest day", "distance": 0.0, "notes": "Rest and recover"}]', 'typical_week_training_review': "Your current training schedule shows short runs spread throughout the week, with a longer run on Satu

In [14]:
from src.email_manager import new_training_week_to_html, send_email
from src.lambda_function import get_athlete_full_name


send_email(
    to={"email": user.email, "name": get_athlete_full_name(strava_client)},
    subject="🏃‍♂️🎯 Just Fixed Your Jawn 🫡",
    html_content=new_training_week_to_html(training_week_with_coaching),
)

{'message_id': '<202409022240.32468387644@smtp-relay.mailin.fr>',
 'message_ids': None}